# Testing bot classifier



Human users from the 'verified-2019' dataset at: \
https://botometer.osome.iu.edu/bot-repository/datasets.html

Bots from Russia IRA dataset and inspiration for code: \
https://www.kaggle.com/datasets/paultimothymooney/russian-political-influence-campaigns \
https://www.kaggle.com/code/kmader/from-hate-speech-to-russian-bot-tweets/notebook

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import Conv1D, GlobalMaxPool1D, Dropout, concatenate
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from IPython.display import clear_output

Using TensorFlow backend.
/Users/ms731/opt/anaconda3/envs/tensorflow-py3.7.9/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/ms731/opt/anaconda3/envs/tensorflow-py3.7.9/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/ms731/opt/anaconda3/envs/tensorflow-py3.7.9/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype

In [2]:
# define network parameters
max_features = 20000
maxlen = 100

In [3]:
russian_troll_tweets1 = pd.read_csv('data/IRA_tweets/IRAhandle_tweets_1.csv')
russian_troll_tweets7 = pd.read_csv('data/IRA_tweets/IRAhandle_tweets_7.csv')
list_sentences_bots = russian_troll_tweets1["content"].fillna("Invalid").values

human_tweets = pd.read_csv('data/tweets_human_200peruser.csv')
list_sentences_humans = human_tweets["Text"].fillna("Invalid").values

list_sentences_train = np.concatenate((list_sentences_humans, list_sentences_bots), axis = 0)

/Users/ms731/opt/anaconda3/envs/tensorflow-py3.7.9/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3417: DtypeWarning: Columns (0,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))

# train data
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
x = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)

#y = np.zeros(np.shape(x_train)[0])
#y = to_categorical(y_train)
y_humans = np.zeros((len(list_sentences_humans)))
y_bots = np.ones((len(list_sentences_bots)))
y = np.concatenate((y_humans, y_bots), axis = 0)
y = to_categorical(y)


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)



In [18]:
"""
#Just me testing what Embedding does
from keras.models import Sequential
model1 = Sequential()
model1.add(Embedding(5, 2, input_length=5))

input_array = np.random.randint(5, size=(1, 5))

model1.compile('rmsprop', 'mse')
output_array = model1.predict(input_array)
"""

In [5]:
print(input_array)
print(output_array)

NameError: name 'input_array' is not defined

In [6]:
def build_model(conv_layers = 2, max_dilation_rate = 4):
    embed_size = 128
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = Dropout(0.25)(x)
    x = Conv1D(2*embed_size, 
                   kernel_size = 3)(x)
    prefilt_x = Conv1D(2*embed_size, 
                   kernel_size = 3)(x)
    out_conv = []
    # dilation rate lets us use ngrams and skip grams to process 
    for dilation_rate in range(max_dilation_rate):
        x = prefilt_x
        for i in range(3):
            x = Conv1D(32*2**(i), 
                       kernel_size = 3, 
                       dilation_rate = 2**dilation_rate)(x)    
        out_conv += [Dropout(0.5)(GlobalMaxPool1D()(x))]
    x = concatenate(out_conv, axis = -1)    
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(2, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['binary_accuracy'])

    return model

model = build_model()
model.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 128)     2560000     input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 100, 128)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 98, 256)      98560       dropout_1[0][0]                 

In [7]:
x_train_short = x_train[:20000]
y_train_short = y_train[:20000]
x_test_short = x_test[:2000]
y_test_short = y_test[:2000]

In [8]:
load_weights = True
file_path="weights.hdf5"
if load_weights is True:
    model.load_weights(file_path)
    
else:
    batch_size = 512
    epochs = 3

    checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    early = EarlyStopping(monitor="val_loss", mode="min", patience=20)

    callbacks_list = [checkpoint, early] #early
    model.fit(x_train_short, y_train_short, 
              batch_size=batch_size, 
              epochs=epochs, 
              validation_split=0.1, 
              callbacks=callbacks_list)
    model.load_weights(file_path)
    #clear_output()

In [9]:
y_pred = model.predict(x_test_short, batch_size=1024, verbose=True)
y_pred_rounded = np.round(y_pred)


2000/2000 [==============================] - 5s 3ms/step


In [10]:
y_pred_rounded.shape

(2000, 2)

In [11]:
from sklearn.metrics import accuracy_score
y_test_short_decoded = np.argmax(y_test_short, axis=-1)
print("Acc", accuracy_score(y_test_short, y_pred_rounded[:2000]))
from sklearn.metrics import confusion_matrix

y_pred_rounded_decoded = np.argmax(y_pred_rounded, axis=-1)
species = np.array(y_test_short_decoded)
predictions = np.array(y_pred_rounded_decoded)
confusion_matrix = confusion_matrix(species, predictions)

print(confusion_matrix)

Acc 0.863
[[897 114]
 [158 831]]


Try with new data

In [12]:
peace_tweets = pd.read_csv('data/tweets_raw_en_df_peacetalks.csv')

list_sentences_peace = peace_tweets["Text"].fillna("Invalid").values 
#tokenizer.fit_on_texts(list(list_sentences_peace))

# train data
list_tokenized_peace = tokenizer.texts_to_sequences(list_sentences_peace)
x_peace = sequence.pad_sequences(list_tokenized_peace, maxlen=maxlen)

x_peace_short = x_peace[:2000]

y_pred_peace = model.predict(x_peace_short, batch_size=1024, verbose=True)
y_pred_peace_decoded = y_pred_peace[:,1]

383/383 [==============================] - 1s 2ms/step


In [13]:
peace_tweets_scores = peace_tweets.copy()

peace_tweets_scores['Bot scores'] = y_pred_peace_decoded

In [14]:
# Just pandas display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/Users/ms731/opt/anaconda3/envs/tensorflow-py3.7.9/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """


In [19]:
peace_tweets_scores

,Unnamed: 0,Datetime,Tweet Id,Text,Username,Like Count,Display Name,Language,User Created,Bot scores
0,0,2022-03-27 23:05:28+00:00,1508218678000508934,Zelensky says Ukraine ready to discuss neutrality in peace talks with Russia https://t.co/fCO8YXdxYi,onej258,0,WSS News,en,2022-02-20,0.502812
1,1,2022-03-27 23:01:13+00:00,1508217608469979138,🙌 &lt;&lt;Zelensky says Ukraine ready to discuss neutrality in peace talks with Russia https://t.co/GDOouvYZ1c&gt;&gt; - Tweet from: @FT #Russia #Ukraine #Zelensky,VisitoryNews,0,Visitory News,en,2022-02-23,0.514012
2,2,2022-03-27 22:27:16+00:00,1508209062374215687,"The next round of in-person peace talks between Ukraine 🇺🇦 and Russia 🇷🇺 will take place on Monday in Istanbul, Turkey, according to a Ukrainian negotiator.",NewsExpert20,0,NewsExpert20,en,2022-03-05,0.509928
3,3,2022-03-27 21:59:00+00:00,1508201948801032194,Russian and Ukrainian delegations to hold new round of peace talks\n\n#RussiaUkraineWar #Ukraine #Russia \nhttps://t.co/TuKzi5KP5g,WeForNews2,0,WeForNews,en,2022-01-13,0.508938
4,4,2022-03-27 20:48:12+00:00,1508184132374966276,"Russia, Ukraine to hold new round of peace talks – Investing NewsEverything https://t.co/C0oYP8nYYV",INewseverything,0,Investing Newseverything,en,2022-02-22,0.504540
5,5,2022-03-27 20:47:04+00:00,1508183848177262593,"@MBOEGV @TheDaneChris as in Libya Iraq..? Ukraine is next door to Russia , but hate's Russia so much to want bases to destroy Russia that horrible leader, Ukraine has got no peace talks he wants on going genocide and ww3",jayzheather7,0,jayzheather7,en,2022-03-02,0.513611
6,6,2022-03-27 20:45:47+00:00,1508183524687425536,@Ukraine You must not let the Russian aggressors take any part of Ukraine when it comes to peace talks and you must ask for the safe return of Ukranians who were deported to Russia!!,Navneet131061,0,Navneet Sharma,en,2022-02-24,0.509952
7,7,2022-03-27 20:05:37+00:00,1508173414569283591,Venue chosen for Russia-Ukraine peace talks — RT World News https://t.co/g5jJ3EAtGh,AngusMemes4,0,𝔸𝕟𝕘𝕦𝕤𝕄𝕖𝕞𝕖𝕤,en,2022-03-03,0.503878
8,8,2022-03-27 20:01:27+00:00,1508172367863992329,"President's Office says Ukraine, Russia foreign ministers ready to meet for peace talks https://t.co/KSwjV3PCzO",JustynaTosiek,0,Justyn4321,en,2022-02-27,0.504660
9,9,2022-03-27 19:54:38+00:00,1508170652972421121,"@Kasparov63 The fact that he thinks he can still have meaning negotiations with Putin is asinine. Putin has shown many times that he doesn’t want negotiations and peace talks. He wants Ukraine under his control, no matter the cost. Macron’s time has come and gone in this war.",TheRealUbereek,2,Ubereek 🇺🇸🇺🇦🌻,en,2022-02-26,0.507940


In [15]:
peace_tweets_scores.loc[np.where((y_pred_peace_decoded > 0.75))[0]]

,Unnamed: 0,Datetime,Tweet Id,Text,Username,Like Count,Display Name,Language,User Created,Bot scores
56,56,2022-03-27 22:55:32+00:00,1508216176748466180,@RALee85 May God remember his peace in Ukraine,RotichR11731394,4,Rotich Robert,en,2022-02-24,0.898852
185,185,2022-03-27 18:07:47+00:00,1508143760324153351,BREAKING: Istanbul will host next round of Russia-Ukraine talks,rusukr321,0,Russia/Ukraine Updates,en,2022-03-01,0.767694
187,187,2022-03-27 18:03:46+00:00,1508142749878521857,BREAKING: Istanbul will host next round of Russia-Ukraine talks.,Breaking_TV_,2,Breaking TV,en,2022-02-08,0.767694
188,188,2022-03-27 18:01:31+00:00,1508142183668490249,BREAKING: Istanbul will host next round of Russia-Ukraine talks,GBNfeed,1,GBN,en,2022-03-05,0.767694
348,348,2022-03-27 18:58:54+00:00,1508156626058031115,@BeckyCNN @michaelh992 @ikalin1 Vicious circle of talks with Russia,tweet4anna,0,Anna 🇺🇦,en,2022-02-10,0.781625


In [19]:
ukraine_tweets = pd.read_csv('data/tweets_raw_en_df_ukraine_feb-march.csv')
ukraine_tweets_feb_march = pd.read_csv('data/tweets_raw_en_df_individual_bot_exploration_ukraine_feb-march1800.csv')
ukraine_tweets_feb_march = ukraine_tweets_feb_march[:1800]
rugby_tweets_feb_march = pd.read_csv('data/tweets_raw_en_df_individual_bot_exploration_rugby_march1800.csv')
rugby_tweets_feb_march = rugby_tweets_feb_march[:1800]

list_sentences_ukraine = ukraine_tweets["Text"].fillna("Invalid").values 
list_sentences_ukraine_feb_march = ukraine_tweets_feb_march["Text"].fillna("Invalid").values 
list_sentences_rugby_feb_march = rugby_tweets_feb_march["Text"].fillna("Invalid").values 

# train data
list_tokenized_ukraine = tokenizer.texts_to_sequences(list_sentences_ukraine)
x_ukraine = sequence.pad_sequences(list_tokenized_ukraine, maxlen=maxlen)
list_tokenized_ukraine_feb_march = tokenizer.texts_to_sequences(list_sentences_ukraine_feb_march)
x_ukraine_feb_march = sequence.pad_sequences(list_tokenized_ukraine_feb_march, maxlen=maxlen)
list_tokenized_rugby_feb_march = tokenizer.texts_to_sequences(list_sentences_rugby_feb_march)
x_rugby_feb_march = sequence.pad_sequences(list_tokenized_rugby_feb_march, maxlen=maxlen)


y_pred_ukraine = model.predict(x_ukraine, batch_size=1024, verbose=True)
y_pred_ukraine_decoded = y_pred_ukraine[:,1]
y_pred_ukraine_feb_march = model.predict(x_ukraine_feb_march, batch_size=1024, verbose=True)
y_pred_ukraine_feb_march_decoded = y_pred_ukraine_feb_march[:,1]
y_pred_rugby_feb_march = model.predict(x_rugby_feb_march, batch_size=1024, verbose=True)
y_pred_rugby_feb_march_decoded = y_pred_rugby_feb_march[:,1]

1800/1800 [==============================] - 4s 2ms/step


In [20]:
ukraine_tweets_scores = ukraine_tweets.copy()
ukraine_tweets_scores['Bot scores'] = y_pred_ukraine_decoded
ukraine_tweets_scores_feb_march = ukraine_tweets_feb_march.copy()
ukraine_tweets_scores_feb_march['Bot scores'] = y_pred_ukraine_feb_march_decoded
rugby_tweets_scores_feb_march = rugby_tweets_feb_march.copy()
rugby_tweets_scores_feb_march['Bot scores'] = y_pred_rugby_feb_march_decoded

In [21]:
ukraine_tweets_scores.loc[np.where((y_pred_ukraine_decoded > 0.75))[0]]

,Unnamed: 0,Datetime,Tweet Id,Text,Username,Like Count,Display Name,Language,User Created,Bot scores
84,84,2022-02-19 23:56:57+00:00,1495185670758879241,"@TeachSUFC Makes sense. Ukraine's armed forces, tanks, planes, ships far outnumber Russia's military.............",laakr1,1,Laak 🇺🇦🇪🇺🇳🇱😷💉💪,en,2011-04-01,0.942965
143,143,2022-02-19 23:55:22+00:00,1495185272530771976,@RonFilipkowski @JDVance1 should lose his Senate race as people do care about Russia invading the Ukraine.,MollyBrown28,0,Molly Brown,en,2011-11-01,0.924052
220,220,2022-02-19 23:52:36+00:00,1495184575777099780,"Trump, aka Putin's Puppet, is cheering for the Authoritarian Dictator; #PATHETIC ...\n\nRT @RpsAgainstTrump: Why hasn’t Donald Trump said a word about Russia’s aggression against Ukraine?",purpleH2Oski,0,Doug Goodman,en,2009-03-12,0.943566
289,289,2022-02-19 23:49:52+00:00,1495183888251080709,@POTUS Ukraine has a government and military controlled by Nazi supporters who are literally terrorists. Russia should invade these Nazis.,MoreWillF,0,Will,en,2021-09-02,0.910871
344,344,2022-02-19 23:48:10+00:00,1495183458532053005,Is MSM seriously trying to blame high gas prices on Russia/Ukraine? 🤣🤡🌎,TheBenTheBoca,2,Ben “Independent Thinker” Ray 121,en,2011-07-27,0.912015
414,414,2022-02-19 23:45:48+00:00,1495182865327345670,Does anybody knows what are those [Z] on the russian hardware? #russia #Ukraine,hcdigo,1,Rodrigo Vargas,en,2021-12-23,0.844372
598,598,2022-02-20 23:56:27+00:00,1495547931419549698,"*WW2 end\n\n*1949 NATO start\n\n*USSR fall\n\n*'99 Baltics join\n\n*Russia warns\n\n*'04 Lat, Lith, Est join\n\n*Rus warns\n\n*'08 NATO allude to Gerogia Ukraine\n\n*'08 Rus invades Georgia\n\n*'09 US/Rus Reset (fails '14)\n\n*'14 Rus invades Crimea/Ukr\n\n...\n\n*'22 Rus invades Ukr\n\nHow it goes, IMHO",OphirGottlieb,34,Ophir Gottlieb,en,2009-12-28,0.802430
604,604,2022-02-20 23:56:19+00:00,1495547896057454597,US warns citizens of terror attacks in Russia as tensions rise with Ukraine https://t.co/yjg4pN29QU,NaijaNewsWire,0,Naija News Agency,en,2014-07-01,0.945681
734,734,2022-02-20 23:52:20+00:00,1495546895552630785,That's the truth.\nhttps://t.co/4DaTX0Nkw7\n\n#Russia #USA #ukraine #NATO,IShamsan,1,I.Shamsan,en,2010-08-27,0.782825
834,834,2022-02-20 23:48:02+00:00,1495545811618443273,"@cnnbrk If anybody start a war, it won't be Ukraine, or Russia!!!!!",darksideofthemo,49,Strange things..👻🇧🇻,en,2011-11-04,0.937404


In [22]:
ukraine_tweets_scores_feb_march.loc[np.where((y_pred_ukraine_feb_march_decoded > 0.75))[0]]

,Unnamed: 0,Datetime,Tweet Id,Text,Username,Like Count,Display Name,Language,User Created,Astroturf,Fake follower,Financial,Other,Overall,Self declared,Spammer,Cap,Bot scores
67,67,2022-02-19 23:56:57+00:00,1495185670758879241,"@TeachSUFC Makes sense. Ukraine's armed forces, tanks, planes, ships far outnumber Russia's military.............",laakr1,1,Laak 🇺🇦🇪🇺🇳🇱😷💉💪,en,2011-04-01,0.11,0.04,0.01,0.36,0.34,0.01,0.07,0.787400,0.942965
110,110,2022-02-19 23:55:22+00:00,1495185272530771976,@RonFilipkowski @JDVance1 should lose his Senate race as people do care about Russia invading the Ukraine.,MollyBrown28,0,Molly Brown,en,2011-11-01,0.17,0.05,0.04,0.21,0.03,0.00,0.00,0.278726,0.924052
176,176,2022-02-19 23:52:36+00:00,1495184575777099780,"Trump, aka Putin's Puppet, is cheering for the Authoritarian Dictator; #PATHETIC ...\n\nRT @RpsAgainstTrump: Why hasn’t Donald Trump said a word about Russia’s aggression against Ukraine?",purpleH2Oski,0,Doug Goodman,en,2009-03-12,0.26,0.20,0.02,0.40,0.35,0.09,0.03,0.789229,0.943566
232,232,2022-02-19 23:49:52+00:00,1495183888251080709,@POTUS Ukraine has a government and military controlled by Nazi supporters who are literally terrorists. Russia should invade these Nazis.,MoreWillF,0,Will,en,2021-09-02,0.05,0.15,0.02,0.12,0.21,0.06,0.15,0.705338,0.910871
276,276,2022-02-19 23:48:10+00:00,1495183458532053005,Is MSM seriously trying to blame high gas prices on Russia/Ukraine? 🤣🤡🌎,TheBenTheBoca,2,Ben “Independent Thinker” Ray 121,en,2011-07-27,0.35,0.14,0.00,0.11,0.05,0.01,0.01,0.334231,0.912015
337,337,2022-02-19 23:45:48+00:00,1495182865327345670,Does anybody knows what are those [Z] on the russian hardware? #russia #Ukraine,hcdigo,1,Rodrigo Vargas,en,2021-12-23,0.04,0.52,0.02,0.56,0.56,0.08,0.22,0.796995,0.844372
487,487,2022-02-20 23:56:27+00:00,1495547931419549698,"*WW2 end\n\n*1949 NATO start\n\n*USSR fall\n\n*'99 Baltics join\n\n*Russia warns\n\n*'04 Lat, Lith, Est join\n\n*Rus warns\n\n*'08 NATO allude to Gerogia Ukraine\n\n*'08 Rus invades Georgia\n\n*'09 US/Rus Reset (fails '14)\n\n*'14 Rus invades Crimea/Ukr\n\n...\n\n*'22 Rus invades Ukr\n\nHow it goes, IMHO",OphirGottlieb,34,Ophir Gottlieb,en,2009-12-28,0.11,0.01,0.00,0.19,0.09,0.00,0.00,0.447947,0.802430
598,598,2022-02-20 23:52:20+00:00,1495546895552630785,That's the truth.\nhttps://t.co/4DaTX0Nkw7\n\n#Russia #USA #ukraine #NATO,IShamsan,1,I.Shamsan,en,2010-08-27,0.02,0.06,0.01,0.06,0.05,0.00,0.00,0.334231,0.782825
684,684,2022-02-20 23:48:02+00:00,1495545811618443273,"@cnnbrk If anybody start a war, it won't be Ukraine, or Russia!!!!!",darksideofthemo,49,Strange things..👻🇧🇻,en,2011-11-04,0.29,0.09,0.03,0.15,0.09,0.05,0.00,0.447947,0.937404
837,837,2022-02-21 23:59:46+00:00,1495911153591099396,Trump campaign gutted GOP’s anti-Russia stance on Ukraine. https://t.co/cKaMlaiVId,DianaCialino,0,Diana Manister,en,2012-03-07,0.95,0.10,0.00,0.33,0.95,0.00,0.00,0.907798,0.805094


In [23]:
rugby_tweets_scores_feb_march.loc[np.where((y_pred_rugby_feb_march_decoded > 0.75))[0]]

,Unnamed: 0,Datetime,Tweet Id,Text,Username,Like Count,Display Name,Language,User Created,Astroturf,Fake follower,Financial,Other,Overall,Self declared,Spammer,Cap,Bot scores
6,6,2022-03-01 23:55:30+00:00,1.498809e+18,Former rugby league star Manu Vatuvei sentenced on meth import charge https://t.co/8w8XSDAH79,MrHilariouskiwi,7,Mr. Kiwi,en,2019-03-05,0.25,0.09,0.00,0.11,0.03,0.00,0.00,0.278726,0.923046
16,16,2022-03-01 23:51:33+00:00,1.498808e+18,Former rugby league star Manu Vatuvei sentenced on meth import charge https://t.co/V4eViO0TP6,aakefu,4,Aminiasi Kefu,en,2014-07-17,0.12,0.13,0.15,0.44,0.18,0.00,0.00,0.659646,0.923046
22,22,2022-03-01 23:49:55+00:00,1.498808e+18,‘They torched our clubhouse’… but Sicilian rugby team won’t let mafia win https://t.co/tPhMh1rte2,stradedellest,0,Giuseppe Acconcia,en,2012-05-29,0.46,0.42,0.04,0.47,0.45,0.01,0.00,0.796647,0.839031
51,51,2022-03-01 23:36:34+00:00,1.498804e+18,Premiership Rugby promotion blocked for Ealing Trailfinders and Doncaster Knights by RFU https://t.co/4SPw0BOQL6,WorldRugbyDouga,0,ラグビー動画まとめ,en,2015-09-25,0.18,0.46,0.03,0.83,0.83,0.60,0.20,0.824092,0.810925
140,140,2022-03-01 23:01:51+00:00,1.498796e+18,Ireland: Andy Farrell releases 11 players to play in United Rugby Championship https://t.co/bdPfT2lucG,WorldRugbyDouga,0,ラグビー動画まとめ,en,2015-09-25,0.18,0.46,0.03,0.83,0.83,0.60,0.20,0.824092,0.784472
168,168,2022-03-01 22:53:21+00:00,1.498794e+18,#rugby history today 01/03 in 1997 : Scotland 38-10 Ireland - IRFU lose in Murrayfield in 1997 rugby test match https://t.co/g9r982TMKS #sru #irfu,TicketsRugby,0,Tickets Rugby,en,2009-01-31,0.14,0.40,0.04,0.86,0.86,0.66,0.35,0.839586,0.893038
230,230,2022-03-01 22:33:55+00:00,1.498789e+18,Army vs Navy rugby next month…. Maybe I can finally get a chance to go,JessicaRose0306,0,Jessica Rose 🌹,en,2013-06-24,0.05,0.10,0.00,0.13,0.07,0.03,0.05,0.391216,0.946923
241,241,2022-03-01 22:32:00+00:00,1.498788e+18,Shaun Wane warns England stars in Rugby League World Cup year as training squad named | @DCravenRL \nhttps://t.co/X87ARnk5Mu https://t.co/UebHHSqMUh,MirrorSport,1,Mirror Sport,en,2010-01-22,0.25,0.10,0.01,0.59,0.59,0.04,0.00,0.796791,0.952422
263,263,2022-03-01 23:03:06+00:00,1.498796e+18,Wales: Alun Wyn Jones rejoins Six Nations squad as he continues shoulder rehabilitation https://t.co/WQ6NRLfNOL,WorldRugbyDouga,0,ラグビー動画まとめ,en,2015-09-25,0.18,0.46,0.03,0.83,0.83,0.60,0.20,0.824092,0.812848
264,264,2022-03-01 23:01:21+00:00,1.498796e+18,Six Nations: Wales’ Nick Tompkins calls for consistency ahead of France Test https://t.co/5QcfkzLBEn,WorldRugbyDouga,0,ラグビー動画まとめ,en,2015-09-25,0.18,0.46,0.03,0.83,0.83,0.60,0.20,0.824092,0.826858


In [54]:
ukraine_tweets_scores_feb_march

,Unnamed: 0,Datetime,Tweet Id,Text,Username,Like Count,Display Name,Language,User Created,Astroturf,Fake follower,Financial,Other,Overall,Self declared,Spammer,Cap,Bot scores
0,0,2022-02-19 23:59:56+00:00,1495186419689693187,So russia has the country of Ukraine🇺🇦 surroun...,MistaWilson513,0,Mista Wilson513,en,2022-01-23,0.11,0.26,0.06,0.29,0.25,0.01,0.05,0.747084,0.000449
1,1,2022-02-19 23:59:54+00:00,1495186412970430464,"@nfseverin1 @RichardHanania You say, stop tell...",Jfweston78,0,Jay W 🇺🇲👨‍🔬💻💹〽️,en,2011-01-05,0.03,0.03,0.00,0.06,0.03,0.01,0.00,0.278726,0.029233
2,2,2022-02-19 23:59:52+00:00,1495186402409078796,39 Ukrainian towns and villages were shelled b...,IntelRogue,1,Intel Rogue,en,2019-03-01,0.27,0.23,0.00,0.34,0.25,0.00,0.02,0.747084,0.020990
3,3,2022-02-19 23:59:41+00:00,1495186358549336064,BBC News - Ukraine: Russia plans biggest war i...,ukprepperlife,0,ukprepper.life,en,2020-09-29,0.11,0.26,0.03,0.22,0.14,0.01,0.03,0.577886,0.036123
4,4,2022-02-19 23:59:41+00:00,1495186357664235526,@ByeByeBoris @KI33RAN @GaryLineker Russia has ...,Kev_____l,0,"Kevin one of the 17,410,742",en,2009-07-02,0.42,0.06,0.00,0.27,0.12,0.00,0.00,0.528866,0.000791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,1795,2022-02-23 23:57:26+00:00,1496635341641498627,Next two hours will be interesting to say the ...,Chronolxgical,5,im back,en,2016-06-21,0.13,0.03,0.01,0.28,0.14,0.00,0.00,0.577886,0.064091
1796,1796,2022-02-23 23:57:25+00:00,1496635339502407681,Here is a Russian's take on what Americans do ...,alexnazaryan,65,Alexander Nazaryan,en,2009-07-25,0.33,0.09,0.00,0.13,0.06,0.00,0.01,0.362646,0.038143
1797,1797,2022-02-23 23:57:24+00:00,1496635334448193536,@tonydoobie @Ellahbie @paulkrugman At the time...,Margare19954289,0,Margaret Baker,en,2019-12-11,0.48,0.08,0.00,0.20,0.20,0.00,0.01,0.691580,0.000444
1798,1798,2022-02-23 23:57:24+00:00,1496635333945049099,someone needs to do a russia ukraine war in hoi4,GeniusCalibur,2,Genius SoulCalibur Facts,en,2021-04-08,0.23,0.39,0.01,0.44,0.38,0.10,0.17,0.793087,0.026808


In [24]:
ukraine_tweets_scores_feb_march.loc[np.where((y_pred_ukraine_feb_march_decoded > 0.75) & (ukraine_tweets_scores_feb_march['Overall'] > 0.6))[0]]

,Unnamed: 0,Datetime,Tweet Id,Text,Username,Like Count,Display Name,Language,User Created,Astroturf,Fake follower,Financial,Other,Overall,Self declared,Spammer,Cap,Bot scores
837,837,2022-02-21 23:59:46+00:00,1495911153591099396,Trump campaign gutted GOP’s anti-Russia stance on Ukraine. https://t.co/cKaMlaiVId,DianaCialino,0,Diana Manister,en,2012-03-07,0.95,0.10,0.00,0.33,0.95,0.00,0.00,0.907798,0.805094
1038,1038,2022-02-21 23:57:21+00:00,1495910546541096976,"@aravosis @chipfranklin I trump were potus, russia wouldn't be invading Ukraine, they'd be invading the USA",bean_joyanna,1,Joyanna Bean,en,2020-01-31,0.89,0.10,0.00,0.22,0.89,0.00,0.00,0.859506,0.907380
1116,1116,2022-02-21 23:56:26+00:00,1495910313803358210,"GLOBAL MARKETS-Oil leaps, stocks slide as Russia moves on Ukraine https://t.co/WPWnQs0wKK",dev_discourse,2,Devdiscourse,en,2017-12-05,0.15,0.40,0.02,0.82,0.82,0.49,0.05,0.819915,0.910504
1646,1646,2022-02-22 23:52:44+00:00,1496271773758808068,@mikepompeo Trump paved the way for Russia to invade the Ukraine.,Ripples04270961,0,Ripple,en,2020-12-20,0.75,0.19,0.03,0.39,0.75,0.00,0.01,0.802148,0.839207
1709,1709,2022-02-23 23:59:03+00:00,1496635747725496322,'Can Russia Actually Control the Entire Landmass of Ukraine?' https://t.co/4oJBcmNwRB https://t.co/OmyLSEarRx,Mbrizzw,0,Monique Brizz-Walker,en,2012-06-12,0.02,0.46,0.04,0.98,0.98,0.76,0.52,0.930754,0.968004
1765,1765,2022-02-23 23:58:05+00:00,1496635505185673217,World changing: 'Can Russia Actually Control the Entire Landmass of Ukraine?' https://t.co/jHUm21tuiA Cheer up!,kawanchi,0,kawanchi obuchi,en,2010-11-10,0.11,0.46,0.09,0.86,0.86,0.81,0.34,0.839586,0.948273


In [25]:
rugby_tweets_scores_feb_march.loc[np.where((y_pred_rugby_feb_march_decoded > 0.75) & (rugby_tweets_scores_feb_march['Overall'] > 0.6))[0]]

,Unnamed: 0,Datetime,Tweet Id,Text,Username,Like Count,Display Name,Language,User Created,Astroturf,Fake follower,Financial,Other,Overall,Self declared,Spammer,Cap,Bot scores
51,51,2022-03-01 23:36:34+00:00,1.498804e+18,Premiership Rugby promotion blocked for Ealing Trailfinders and Doncaster Knights by RFU https://t.co/4SPw0BOQL6,WorldRugbyDouga,0,ラグビー動画まとめ,en,2015-09-25,0.18,0.46,0.03,0.83,0.83,0.60,0.20,0.824092,0.810925
140,140,2022-03-01 23:01:51+00:00,1.498796e+18,Ireland: Andy Farrell releases 11 players to play in United Rugby Championship https://t.co/bdPfT2lucG,WorldRugbyDouga,0,ラグビー動画まとめ,en,2015-09-25,0.18,0.46,0.03,0.83,0.83,0.60,0.20,0.824092,0.784472
168,168,2022-03-01 22:53:21+00:00,1.498794e+18,#rugby history today 01/03 in 1997 : Scotland 38-10 Ireland - IRFU lose in Murrayfield in 1997 rugby test match https://t.co/g9r982TMKS #sru #irfu,TicketsRugby,0,Tickets Rugby,en,2009-01-31,0.14,0.40,0.04,0.86,0.86,0.66,0.35,0.839586,0.893038
263,263,2022-03-01 23:03:06+00:00,1.498796e+18,Wales: Alun Wyn Jones rejoins Six Nations squad as he continues shoulder rehabilitation https://t.co/WQ6NRLfNOL,WorldRugbyDouga,0,ラグビー動画まとめ,en,2015-09-25,0.18,0.46,0.03,0.83,0.83,0.60,0.20,0.824092,0.812848
264,264,2022-03-01 23:01:21+00:00,1.498796e+18,Six Nations: Wales’ Nick Tompkins calls for consistency ahead of France Test https://t.co/5QcfkzLBEn,WorldRugbyDouga,0,ラグビー動画まとめ,en,2015-09-25,0.18,0.46,0.03,0.83,0.83,0.60,0.20,0.824092,0.826858
320,320,2022-03-01 20:19:44+00:00,1.498755e+18,#SPOTLIGHT: @WelshRugbyUnion stalwart returns to @SixNationsRugby squad! \nhttps://t.co/H9O1tIaymp,rugby365com,0,rugby365.com,en,2009-05-18,0.28,0.46,0.00,0.73,0.73,0.35,0.00,0.799836,0.898034
464,464,2022-03-01 14:44:40+00:00,1.498671e+18,Six Nations: Wales captain Alun Wyn Jones links up with squad to continue rehab from shoulder injury - Sky Sports https://t.co/loI4YboSAH,ruckmeslowly,0,Ruck On,en,2011-10-05,0.03,0.40,0.01,0.91,0.91,0.74,0.27,0.874858,0.767781
538,538,2022-03-02 23:42:07+00:00,1.499168e+18,Promising rugby player avoids convictions after 'bizarre' assault https://t.co/WvTVVkbxXb,WorldRugbyDouga,0,ラグビー動画まとめ,en,2015-09-25,0.18,0.46,0.03,0.83,0.83,0.60,0.20,0.824092,0.848880
655,655,2022-03-02 22:50:03+00:00,1.499155e+18,Prince George Accompanies Kate Middleton and Prince William to Rugby Match https://t.co/igUttH4MEL,Bewee_store,0,Bewee Store,en,2021-07-23,0.00,0.68,0.08,0.90,0.98,0.98,0.74,0.930754,0.919285
869,869,2022-03-02 20:47:00+00:00,1.499124e+18,"Saracens give update on ""world-class"" Owen Farrell's return date amid Six Nations absence\n\nhttps://t.co/OOAbWjlgzn #GallagherPrem https://t.co/WxZfFpQ4f7",MirrorRugbyU,0,Mirror Rugby Union,en,2009-06-04,0.14,0.22,0.00,0.74,0.74,0.56,0.06,0.800880,0.918169


In [26]:
np.where(ukraine_tweets_scores_feb_march['Astroturf'] > 0.75)[0]

array([  15,   25,   40,   83,  108,  111,  143,  151,  388,  419,  432,
        455,  478,  493,  522,  526,  527,  579,  584,  703,  706,  724,
        778,  779,  785,  837,  853,  862,  878,  952,  959,  992,  995,
       1038, 1092, 1122, 1225, 1335, 1354, 1355, 1363, 1364, 1375, 1378,
       1432, 1451, 1465, 1515, 1527, 1562, 1566, 1635, 1638, 1639, 1672,
       1681, 1726, 1738, 1750])

## Data manipulation investigation

One thing we can experiment with is replacing some of the words in the bot dataset with ones we think are relevant to bots in Ukraine e.g. USA/America -> Russia/Ukraine, Trump -> Putin, or even Trump -> Zelensky if the model picks up on hot-headed opinions even if they are nothing alike. 

Let's find some of the most common words in the bot dataset